In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms

In [5]:
tr = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], 
                        [0.229, 0.224, 0.225])
    ])
train_set = datasets.ImageFolder(r'Post-hurricane\train_another',tr)
val_set   = datasets.ImageFolder(r'Post-hurricane\validation_another',tr)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=4,
                                       shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=4,  
                                       shuffle=True, num_workers=4)
classes = train_set.classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Feature Extraction

In [6]:
model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

C:\Users\Maartin\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Maartin\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [8]:
for epoch in range(25):
    running_loss = 0.0
    train_running_correct = 0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        data, target = data[0].to(device), data[1].to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        _, preds = torch.max(outputs.data, 1)
        train_running_correct += (preds == target).sum().item()
    
    train_loss = running_loss/len(train_loader.dataset)
    train_accuracy = 100. * train_running_correct/len(train_loader.dataset)
    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}')

print('Finished Training')

Train Loss: 0.1220, Train Acc: 80.55
Train Loss: 0.1196, Train Acc: 82.05
Train Loss: 0.1126, Train Acc: 82.78
Train Loss: 0.1158, Train Acc: 83.42
Train Loss: 0.1143, Train Acc: 82.92
Train Loss: 0.1166, Train Acc: 82.84
Train Loss: 0.1179, Train Acc: 83.35
Train Loss: 0.1213, Train Acc: 81.91
Train Loss: 0.1118, Train Acc: 83.14
Train Loss: 0.1190, Train Acc: 82.81
Train Loss: 0.1144, Train Acc: 83.18
Train Loss: 0.1160, Train Acc: 82.75
Train Loss: 0.1123, Train Acc: 83.49
Train Loss: 0.1130, Train Acc: 83.57
Train Loss: 0.1116, Train Acc: 83.40
Train Loss: 0.1187, Train Acc: 83.36
Train Loss: 0.1167, Train Acc: 83.00
Train Loss: 0.1188, Train Acc: 83.69
Train Loss: 0.1109, Train Acc: 83.83
Train Loss: 0.1155, Train Acc: 83.31
Train Loss: 0.1196, Train Acc: 83.08
Train Loss: 0.1221, Train Acc: 82.67
Train Loss: 0.1096, Train Acc: 83.39
Train Loss: 0.1096, Train Acc: 84.07
Train Loss: 0.1099, Train Acc: 83.65
Finished Training


In [48]:
class_correct = list(0. for i in range(2))
class_total = list(0. for i in range(2))
with torch.no_grad():
    for i, data in enumerate(val_loader, 0):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(4):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1
for i in range(2):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of damage : 90 %
Accuracy of no_damage : 80 %


# Transfert Learning A

In [49]:
#dataloader objet PyTorch qui contient les données à partir desquelles les caractéristiques doivent être extraites
#model : un objet modèle de réseau de neurones qui sera utilisé pour extraire les caractéristiques.
#n_sample : un entier qui spécifie le nombre de données à extraire.

#Ensuite, la boucle itère à travers les lots de données du dataloader.
#Pour chaque lot, le modèle passe en mode d'évaluation, ce qui signifie qu'il ne tiendra pas compte de la descente de gradient.
#Les caractéristiques sont extraites en faisant passer le lot d'entrées à travers le modèle et en détachant les valeurs obtenues.
#Les caractéristiques sont converties en numpy array dans features.
#Les étiquettes sont stockées labels.

def extract_features(dataloader, model, n_sample):
    features = []
    labels = []
    i = 0
    for inputs_batch, labels_batch in dataloader:
        model.eval()
        features_batch = model(inputs_batch).detach().numpy()
        features[i * dataloader.batch_size: (i + 1) * dataloader.batch_size] = features_batch
        labels[i * dataloader.batch_size: (i + 1) * dataloader.batch_size] = labels_batch
        i += 1
        if i * dataloader.batch_size >= n_sample:
            break
    return features, labels

In [50]:
model = models.resnet18(pretrained=True)

In [51]:
train_features, train_labels = extract_features(train_loader, model, 10000)
test_features, test_labels = extract_features(val_loader, model, 2000)

Random Forest

In [52]:
from sklearn.ensemble import RandomForestClassifier

# Convertir les caractéristiques en numpy array pour les utiliser avec scikit-learn
# Entraînez un classificateur Random Forest
clf = RandomForestClassifier()
clf.fit(train_features, train_labels)

RandomForestClassifier()

In [1]:
import wandb
wandb.sklearn.plot_feature_importances(model, [‘width’, ‘height, ‘length’])

In [53]:
pred = clf.predict(test_features)

from sklearn.metrics import classification_report
print(classification_report(test_labels, pred))

              precision    recall  f1-score   support

           0       0.91      0.89      0.90      1000
           1       0.90      0.92      0.91      1000

    accuracy                           0.90      2000
   macro avg       0.90      0.90      0.90      2000
weighted avg       0.90      0.90      0.90      2000

